In [2]:
import numpy as np
import pandas as pd
import sqlite3
import tsfresh
import dask.dataframe as dd

In [3]:
# Get path and connection
path_in = "S:\Dehydration_stroke\Team Emerald\Working Data\Preprocessed\Working\Merged.db"
con = sqlite3.connect(path_in)

In [4]:
static = pd.read_sql_query("SELECT * FROM static_predictors", con)
outcomes = pd.read_sql_query("SELECT * FROM outcomes", con)

In [65]:
# Load in extracted timeseries data
df = pd.read_csv('extracted_flowsheet_first24h.csv')
df = df.drop('Unnamed: 0', axis=1)

In [11]:
los = outcomes.sort_values('mrn_csn_pair')['los_ge_7d'].reset_index(drop=True)

In [67]:
# Drop all columns with only 0
df = df.loc[:, (df != 0).any(axis=0)]

In [68]:
# Do tsfresh feature filtering
feature_table = tsfresh.feature_selection.relevance.calculate_relevance_table(df.drop('mrn_csn_pair', axis=1), los, n_jobs=-1)

In [69]:
complete = pd.concat([df.drop('mrn_csn_pair', axis=1).loc[:, feature_table['relevant']], static.sort_values('mrn_csn_pair').reset_index(drop=True)], axis=1)

In [77]:
complete.insert(0, 'LOS', los)
pairs = complete['mrn_csn_pair']
complete = complete.drop('mrn_csn_pair', axis=1)
complete.insert(1, 'mrn_csn_pair', pairs)

In [81]:
complete.to_csv("S:/Dehydration_stroke/Team Emerald/Working Data/Preprocessed/Working/Complete_updated.csv")

In [ ]:
con.close()
con_out.close()